In [ ]:
# --- Cell 0: Interactive File Setup ---
%matplotlib widget

from pathlib import Path

import ipywidgets as widgets
from ipyfilechooser import FileChooser

import opym

print("1. Select your base OME-TIF file (e.g., ...Pos0.ome.tif).")
print("2. Select the Timepoint (T) you want to use for the MIP.")
print("3. Run Cell 1 to calculate the MIP.")

# --- Create the file chooser ---
file_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    filter_pattern="*.ome.tif",
    title="<b>Select Base OME-TIF File:</b>",
)

# --- Slider for Timepoint selection ---
t_slider = widgets.IntSlider(
    description="Timepoint (T):",
    min=0,
    max=199,  # Will be updated by Cell 1
    step=1,
    value=0,
)

display(file_chooser, t_slider)

In [ ]:
# --- Cell 1: Create the MIPs ---
# This cell now calls the `create_mip` function from your package

try:
    file_to_inspect = Path(file_chooser.value)
    t_index = t_slider.value

    # Call the refactored function
    mip_data, vmin, vmax, lazy_data, t_max = opym.create_mip(file_to_inspect, t_index)

    # Update the slider in Cell 0 with the correct max T
    t_slider.max = t_max
    t_slider.value = t_index

    print("\nYou can now run Cell 2 to select ROIs.")

except FileNotFoundError:
    print("❌ ERROR: File not found. Please select a file in Cell 0 and re-run.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 2: Select ROIs Interactively ---
# This cell uses the new ROISelector class from your package

try:
    # This will display the plot and wait for you to draw two ROIs
    selector = opym.interactive_roi_selector(mip_data, vmin, vmax)
except NameError:
    print("❌ ERROR: 'mip_data' not found. Please run Cell 1 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 3: Get ROIs and Auto-Align ---
# This cell gets the ROIs from the selector and calls the align function

try:
    # This call will block until two ROIs are drawn in the plot from Cell 2
    unaligned_rois = selector.get_rois()

    top_roi_unaligned = unaligned_rois[0]
    bottom_roi_unaligned = unaligned_rois[1]

    print("\n--- Auto-Aligning ROIs ---")

    # --- FIX: Call align_rois with mip_data, not lazy_data ---
    bottom_roi_aligned = opym.align_rois(
        mip_data,  # <-- Pass the 2D MIP data
        top_roi_unaligned,
        bottom_roi_unaligned,
    )
    # ---------------------------------------------------------

    # Store the final, aligned ROIs for processing
    top_roi = top_roi_unaligned
    bottom_roi = bottom_roi_aligned

    print("\n✅ Alignment complete. Run Cell 4 to visualize.")

except NameError:
    print(
        "❌ ERROR: 'selector' or 'mip_data' not found. Please run Cells 1 and 2 first."
    )
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 4: Visualize Final Alignment ---
# This cell calls the new visualization function from your package

try:
    opym.visualize_alignment(mip_data, top_roi, bottom_roi, vmin, vmax)
    print("\nRun Cell 5 to save these ROIs to your log file.")
except NameError:
    print("❌ ERROR: Required variables not found. Please run Cells 1, 2, and 3 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 5: Save ROIs to Log File ---
# This cell uses the save_rois_to_log function from your package

try:
    # Derive paths to find the log file location
    paths = opym.derive_paths(file_to_inspect, opym.OutputFormat.ZARR)

    opym.save_rois_to_log(
        log_file=paths.output_log,
        base_file=file_to_inspect,
        top_roi=top_roi,
        bottom_roi=bottom_roi,
    )

    # --- The print statement below was removed ---
    print("You can now use these ROIs for batch processing via the CLI.")

except NameError:
    print("❌ ERROR: Required variables not found. Please run all previous cells.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 6: Run Cropping Job ---

import importlib
import sys
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display

# --- 1. Import the opym package ---
try:
    import opym
    from opym.utils import OutputFormat

    # Reload modules to pick up any changes
    importlib.reload(opym.utils)
    importlib.reload(opym.metadata)
    importlib.reload(opym.core)
    importlib.reload(opym)
    print("Successfully re-loaded opym package.")

    # --- 2. Create Format Selector Widget ---
    output_format_widget = widgets.RadioButtons(
        options=[(f.value, f) for f in OutputFormat],
        description="Output Format:",
        value=OutputFormat.ZARR,  # Default to Zarr
        disabled=False,
    )
    run_button = widgets.Button(
        description="Run Processing Job", button_style="success"
    )
    run_output = widgets.Output()

    display(widgets.VBox([output_format_widget, run_button, run_output]))

    # --- 3. Define global vars to store paths ---
    processing_output_dir: Path | None = None  # <-- ADD THIS
    processing_format: OutputFormat | None = None  # <-- ADD THIS

    # --- 4. Define the function to run on button click ---
    def on_run_button_clicked(b):
        # --- Make variables global ---
        global processing_output_dir, processing_format

        with run_output:
            run_output.clear_output()

            try:
                # --- Store format and derive paths ---
                processing_format = output_format_widget.value
                paths = opym.derive_paths(file_to_inspect, processing_format)  # type: ignore
                processing_output_dir = paths.output_dir

                # --- 4. Call the new package function --
                opym.run_processing_job(
                    base_file=file_to_inspect,
                    top_roi=top_roi,
                    bottom_roi=bottom_roi,
                    output_format=processing_format,  # Use variable # type: ignore
                    cli_log_file=Path("opm_roi_log.json"),
                )

                print("\n--- Processing Job Complete ---")

            except NameError as e:
                print(f"❌ ERROR: A required variable is missing: {e}", file=sys.stderr)
                print("Please ensure you have run Cell 0, 1, 3, and 5 first.")
            except Exception as e:
                print(f"❌ An unexpected error occurred: {e}", file=sys.stderr)

    # --- 5. Link the button to the function ---
    run_button.on_click(on_run_button_clicked)

except ImportError:
    print("❌ ERROR: Could not import opym package.", file=sys.stderr)
    print(
        "Please install it from the repo root: `uv pip install -e .`", file=sys.stderr
    )

In [ ]:
# --- Cell 7: Run PyPetaKit5D Processing ---
# This cell uses the paths defined in Cell 6.

import ipywidgets as widgets
from IPython.display import display

# --- Create Widgets ---
petakit_button = widgets.Button(
    description="Run PetaKit Processing (Deskew/Rotate)", button_style="success"
)
petakit_output = widgets.Output()


# --- Define the function to run on button click ---
def on_petakit_button_clicked(b):
    with petakit_output:
        petakit_output.clear_output()

        try:
            # Check that the previous steps have been run
            if processing_output_dir is None or processing_format is None:
                print(
                    "❌ ERROR: 'processing_output_dir' not set. Please run Cell 6 first"
                )
                return

            # Check that the format is correct for this step
            if processing_format != opym.OutputFormat.TIFF_SERIES:
                print("❌ ERROR: PetaKit step requires TIFF_SERIES_SPLIT_C.")
                print(f"   Current format is: {processing_format.value}")
                print("   Please re-run Cell 6 with the correct format.")
                return

            print(
                f"-- Running PetaKit5D processing for: {processing_output_dir.name} --"
            )

            # Call the function using the path from Cell 6
            opym.run_petakit_processing(
                processing_output_dir,
            )

            print("\n✅ PetaKit5D job finished!")

        except FileNotFoundError as e:
            print("\n❌ ERROR: Could not find required directories or files.")
            print(f"  Details: {e}")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)


# --- Link button and display UI ---
petakit_button.on_click(on_petakit_button_clicked)
display(widgets.VBox([petakit_button, petakit_output]))

In [ ]:
# --- Cell 8: Load Final Processed Data ---
# This cell loads the final deskewed + rotated TIFF series
# from the 'DSR' directory created in Cell 7.

import ipywidgets as widgets
from IPython.display import display
import opym # Make sure opym is imported

# --- Global variable to hold viewer parameters ---
viewer_params = None

# --- Create Widgets ---
load_dsr_button = widgets.Button(
    description="Load Final DSR Data", button_style="primary"
)
load_dsr_output = widgets.Output()


# --- Define the function to run on button click ---
def on_load_dsr_button_clicked(b):
    global viewer_params # Make sure we're assigning to the global var

    with load_dsr_output:
        load_dsr_output.clear_output()

        try:
            # Check that the previous steps have been run
            if processing_output_dir is None:
                print(
                    "❌ ERROR: 'processing_output_dir' not set. Run Cell 6 & 7 first."
                )
                return

            # --- Automatically find the final DSR directory ---
            dsr_data_dir = processing_output_dir / "DSR"

            if not dsr_data_dir.exists():
                print(f"❌ ERROR: DSR directory not found at: {dsr_data_dir}")
                print("   Please run Cell 7 (PetaKit Processing) first.")
                return

            print(f"--- Loading data from: {dsr_data_dir.name} ---")

            # 1. Call the data loader
            (
                get_stack,
                t_min,
                t_max,
                c_min,
                c_max,
                z_max,
                y,
                x,
                base_name,
            ) = opym.load_tiff_series(dsr_data_dir)

            # 2. Store parameters in the global variable
            viewer_params = (get_stack, t_max, z_max, c_max, y, x)

            print(
                f"✅ Data loaded. Shape: T={t_min}-{t_max}, "
                f"Z={z_max + 1}, C={c_min}-{c_max}, Y={y}, X={x}"
            )
            print(f"   Base name: {base_name}")
            print("\nYou can now run the viewer cells below.")

        except FileNotFoundError as e:
            print("\n❌ ERROR: Could not load data.")
            print(f"  Details: {e}")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)


# --- Link button and display UI ---
load_dsr_button.on_click(on_load_dsr_button_clicked)
display(widgets.VBox([load_dsr_button, load_dsr_output]))

In [ ]:
# --- Cell 9: Launch the single-channel viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.single_channel_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 8 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
# --- Cell 10: Launch the composite viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.composite_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 8 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")